In [32]:
import ee
import geemap
from geemap import ml
from sklearn import ensemble
import pandas as pd


# Inicializa la autenticación y la inicialización de Google Earth Engine
ee.Authenticate()
ee.Initialize(project='ee-facuboladerasgee')

In [33]:
# roi = ee.FeatureCollection('projects/ee-facuboladerasgee/assets/Forestaciones_uru')
Map = geemap.Map(center=[0, 0], zoom=2)
Map.add_basemap('SATELLITE')
Map


Map(center=[0, 0], controls=(WidgetControl(options=['position', 'transparent_bg'], widget=SearchDataGUI(childr…

In [3]:
# Capturar el polígono dibujado como ROI
roi = Map.user_roi
coords = roi.coordinates()

print(coords.getInfo())

geom = ee.Geometry.Polygon(coords)
roi = geom

[[[-58.290672, -31.844432], [-58.290672, -31.70492], [-58.109465, -31.70492], [-58.109465, -31.844432], [-58.290672, -31.844432]]]


In [13]:
# Capturar el polígono dibujado como ROI
# roi = Map.user_roi
# coords = roi.coordinates()

# print(coords.getInfo())

# geom = ee.Geometry.Polygon(coords)
# roi = geom

In [ ]:
roi = ee.FeatureCollection("projects/ee-facuboladerasgee/assets/delta_25_06_2021")
roi

In [38]:

# ---------------------------------------------------------------------
# 1) PARAMETERS
# ---------------------------------------------------------------------
EMB_COLLECTION  = 'GOOGLE/SATELLITE_EMBEDDING/V1/ANNUAL'
GEDI_COLLECTION = 'LARSE/GEDI/GEDI04_A_002_MONTHLY'
TARGET_BAND     = 'agbd'   # above-ground biomass density (Mg ha⁻¹)


# ---------------------------------------------------------------------
# 3) YEAR 2: 2024
# ---------------------------------------------------------------------
YEAR = 2024
START = f'{YEAR}-01-01'
END = f'{YEAR}-12-31'

embedding = (ee.ImageCollection(EMB_COLLECTION)
              .filterDate(START, END))

embedding_image = embedding.median().clip(roi)
# ---------------------------------------------------------------------
# Add SRTM DEM to stack
# ---------------------------------------------------------------------
srtm = (ee.Image('USGS/SRTMGL1_003')
          .select('elevation')
          .clip(roi)
          .rename('elevation'))

dw = (ee.ImageCollection('GOOGLE/DYNAMICWORLD/V1')
          .filterBounds(roi)
          .filterDate(START, END)
          .select('label')
          .mode()      # clave para clases
          .clip(roi))

label = dw.rename("label")
# (opcional) derivados topográficos
terrain = ee.Terrain.products(srtm)
slope   = terrain.select('slope').rename('slope')
aspect  = terrain.select('aspect').rename('aspect')

# Stack final: embeddings + DEM (+ slope/aspect si querés)
embedding_image = embedding_image.addBands([srtm, slope, aspect,label])

In [ ]:
embedding_image

In [17]:
Map.addLayer(embedding_image,{},"embedding")
Map.addLayer(embedding_image.select("elevation"),{},"elevation")
Map

Map(bottom=1244238.0, center=[-31.770499512938443, -58.16728591918946], controls=(WidgetControl(options=['posi…

In [18]:
def mask_excluding_built(image, built_id=6):
    dw = (ee.ImageCollection('GOOGLE/DYNAMICWORLD/V1')
          .filterBounds(roi)
          .filterDate(START, END)
          .select('label')
          .mode()      # clave para clases
          .clip(roi))

    not_built_mask = dw.neq(built_id)  # <-- TODO menos built
    return image.updateMask(not_built_mask)


# Obtener los datos GEDI de la altura de dosel
def get_gedi_canopy_height(band_name='rh98', sens_thr=0.9):
    col = (ee.ImageCollection('LARSE/GEDI/GEDI02_A_002_MONTHLY')
        .filterBounds(roi)
        .filterDate(START, END)
    )

    def mask_quality(img):
        m = img.select('degrade_flag').eq(0)
        m = m.And(img.select('quality_flag').eq(1))
        m = m.And(img.select('sensitivity').gte(sens_thr))
        return img.updateMask(m)

    return (col.map(mask_quality)
              .select(band_name)
              .median()
              .toFloat()
              .clip(roi)
              .rename(band_name))


gedi_rh98 = get_gedi_canopy_height('rh95').rename('rh95')


# excluir construido
gedi_masked = mask_excluding_built(gedi_rh98, built_id=6)
image = embedding_image.addBands(gedi_masked.rename('rh98'))


In [19]:
image

In [11]:
Map.addLayer(image.select('rh98'),{},"rh98")
Map

Map(bottom=311224.0, center=[-31.735175219118695, -58.14309778682694], controls=(WidgetControl(options=['posit…

In [20]:
import time

sample = image.sample(
    scale=10,  # Ajusta la escala según sea necesario para tus datos
    region=roi,
    geometries=True  # Incluir geometría de los puntos
)

# Exportar la tabla de muestra a Google Drive en formato CSV
export_task = ee.batch.Export.table.toDrive(
    collection=sample,
    description='costa_de_uruguay_completo_filter95',
    folder='CH-delta',
    fileNamePrefix=f'costa_de_uruguay_completo_filter95',
    fileFormat='CSV'
)


# Iniciar la tarea de exportación
export_task.start()

    # Esperar a que la tarea de exportación se complete
export_task.status()

    # Verificar el estado de la tarea y mostrar un mensaje de éxito
while export_task.active():
    print('Exportación en progreso...')
    time.sleep(30)  # Esperar 30 segundos antes de verificar el estado nuevamente
    
    if export_task.status()['state'] == 'COMPLETED':
        print(f'Exportación completada con éxito.')
    else:
        print(f'Error en la exportación: {export_task.status()}')

Exportación en progreso...
Error en la exportación: {'state': 'RUNNING', 'description': 'costa_de_uruguay_completo_filter95', 'creation_timestamp_ms': 1764075364776, 'update_timestamp_ms': 1764075393435, 'start_timestamp_ms': 1764075371615, 'task_type': 'EXPORT_FEATURES', 'attempt': 1, 'id': '4JM6EVR4TJA5X2KEUJBK5XRC', 'name': 'projects/ee-facuboladerasgee/operations/4JM6EVR4TJA5X2KEUJBK5XRC'}
Exportación en progreso...
Exportación completada con éxito.


# Predicts

In [21]:
feature_names = ['A00', 'A01', 'A02', 'A03', 'A04', 'A05', 'A06', 'A07', 'A08', 'A09', 'A10', 'A11', 'A12', 'A13', 'A14', 'A15', 'A16', 'A17', 'A18', 'A19', 'A20', 'A21', 'A22', 'A23', 'A24', 'A25', 'A26', 'A27', 'A28', 'A29', 'A30', 'A31', 'A32', 'A33', 'A34', 'A35', 'A36', 'A37', 'A38', 'A39', 'A40', 'A41', 'A42', 'A43', 'A44', 'A45', 'A46', 'A47', 'A48', 'A49', 'A50', 'A51', 'A52', 'A53', 'A54', 'A55', 'A56', 'A57', 'A58', 'A59', 'A60', 'A61', 'A62', 'A63', 'aspect', 'elevation', 'label', 'slope']

image = embedding_image.select(feature_names)

In [22]:
image

In [28]:
feature_names = ['A00', 'A01', 'A02', 'A03', 'A04', 'A05', 'A06', 'A07', 'A08', 'A09', 'A10', 'A11', 'A12', 'A13', 'A14', 'A15', 'A16', 'A17', 'A18', 'A19', 'A20', 'A21', 'A22', 'A23', 'A24', 'A25', 'A26', 'A27', 'A28', 'A29', 'A30', 'A31', 'A32', 'A33', 'A34', 'A35', 'A36', 'A37', 'A38', 'A39', 'A40', 'A41', 'A42', 'A43', 'A44', 'A45', 'A46', 'A47', 'A48', 'A49', 'A50', 'A51', 'A52', 'A53', 'A54', 'A55', 'A56', 'A57', 'A58', 'A59', 'A60', 'A61', 'A62', 'A63', 'aspect', 'elevation', 'label', 'slope']

label = "rh98"

user_id = 'users/facuboladerasgee'
# specify asset id where to save trees
# be sure to change  to your ee user name
asset_id = user_id + "/DELTA-CHM_rh95"
asset_id

# read the exported tree feature collection
rf_fc = ee.FeatureCollection(asset_id)

# convert it to a classifier, very similar to the `ml.trees_to_classifier` function
another_classifier = ml.fc_to_classifier(rf_fc)

# classify the image again but with the classifier from the persisted trees
classified = image.select(feature_names).classify(another_classifier)

In [30]:
visPredictedBiomass = {
    'min': 0,
    'max': 30,
    'palette': ['lightyellow', 'lightgreen', 'green', 'darkgreen']
}


Map.centerObject(roi)
Map.addLayer(
    classified,
    visPredictedBiomass,
    "im1",)


Map

Map(bottom=622283.0, center=[-31.774676, -58.2000685], controls=(WidgetControl(options=['position', 'transpare…

In [31]:
# Export 2017
task1 = ee.batch.Export.image.toDrive(
    image=classified,
    description='Prediccion_2024_colon_fix95',
    folder='CH-delta',       # Carpeta en tu Google Drive
    fileNamePrefix='Prediccion_2024_colon_fix95',
    region=roi,
    scale=10,                    # Resolución de 100 metros
    crs='EPSG:4326',              # Puedes ajustar el CRS según tus datos
    maxPixels=1e13
)
task1.start()

print("🚀 Exportaciones lanzadas: revisa la pestaña 'Tasks' en GEE")

🚀 Exportaciones lanzadas: revisa la pestaña 'Tasks' en GEE


# Cargar rf dividido

In [13]:
feature_names = ['A00', 'A01', 'A02', 'A03', 'A04', 'A05', 'A06', 'A07', 'A08', 'A09', 'A10', 'A11', 'A12', 'A13', 'A14', 'A15', 'A16', 'A17', 'A18', 'A19', 'A20', 'A21', 'A22', 'A23', 'A24', 'A25', 'A26', 'A27', 'A28', 'A29', 'A30', 'A31', 'A32', 'A33', 'A34', 'A35', 'A36', 'A37', 'A38', 'A39', 'A40', 'A41', 'A42', 'A43', 'A44', 'A45', 'A46', 'A47', 'A48', 'A49', 'A50', 'A51', 'A52', 'A53', 'A54', 'A55', 'A56', 'A57', 'A58', 'A59', 'A60', 'A61', 'A62', 'A63', 'aspect', 'elevation', 'label', 'slope']


def load_rf_from_parts_auto(base_asset_id, tree_prop='tree', verbose=True):
    """
    Carga automáticamente un modelo RF dividido en partes sin conocer cuántas hay.
    Las partes deben tener formato:
       base_asset_id + "_part_0"
       base_asset_id + "_part_1"
       ...
    Ejemplo:
       users/.../Rf_CHM-FORESTAL_part_0
       users/.../Rf_CHM-FORESTAL_part_1
       ...
    """
    merged = ee.FeatureCollection([])
    i = 0
    parts_loaded = 0

    while True:
        part_id = f"{base_asset_id}_part_{i}"
        try:
            fc_part = ee.FeatureCollection(part_id)
            # intentar obtener tamaño obliga a validar existencia
            fc_part.limit(1).size().getInfo()

            merged = merged.merge(fc_part)
            parts_loaded += 1

            if verbose:
                print(f"✔ Parte {i} cargada: {part_id}")

            i += 1

        except Exception:
            if verbose:
                print(f"❗ No existe la parte {i} → terminado.")
            break

    if parts_loaded == 0:
        raise ValueError("No se detectó ninguna parte del modelo.")

    if verbose:
        print(f"\nTotal partes cargadas: {parts_loaded}")
        print("Reconstruyendo classifier...")

    # reconstruir classifier
    clf = ml.fc_to_classifier(merged).setOutputMode('REGRESSION')
    return clf

# ---- uso ----
base_asset_id = "users/facuboladerasgee/Rf_CHM-uruguay"
another_classifier = load_rf_from_parts_auto(base_asset_id)

classified = image.classify(another_classifier)

✔ Parte 0 cargada: users/facuboladerasgee/Rf_CHM-uruguay_part_0
✔ Parte 1 cargada: users/facuboladerasgee/Rf_CHM-uruguay_part_1
✔ Parte 2 cargada: users/facuboladerasgee/Rf_CHM-uruguay_part_2
✔ Parte 3 cargada: users/facuboladerasgee/Rf_CHM-uruguay_part_3
✔ Parte 4 cargada: users/facuboladerasgee/Rf_CHM-uruguay_part_4
✔ Parte 5 cargada: users/facuboladerasgee/Rf_CHM-uruguay_part_5
✔ Parte 6 cargada: users/facuboladerasgee/Rf_CHM-uruguay_part_6
✔ Parte 7 cargada: users/facuboladerasgee/Rf_CHM-uruguay_part_7
✔ Parte 8 cargada: users/facuboladerasgee/Rf_CHM-uruguay_part_8
✔ Parte 9 cargada: users/facuboladerasgee/Rf_CHM-uruguay_part_9
✔ Parte 10 cargada: users/facuboladerasgee/Rf_CHM-uruguay_part_10
✔ Parte 11 cargada: users/facuboladerasgee/Rf_CHM-uruguay_part_11
✔ Parte 12 cargada: users/facuboladerasgee/Rf_CHM-uruguay_part_12
✔ Parte 13 cargada: users/facuboladerasgee/Rf_CHM-uruguay_part_13
✔ Parte 14 cargada: users/facuboladerasgee/Rf_CHM-uruguay_part_14
❗ No existe la parte 15 → term

In [47]:
# pred = image.classify(another_classifier).rename("pred")
# # Desacoplar grafo grande
# pred_simplified = pred.reproject(pred.projection()).focal_mean(1)

In [46]:
# visPredictedBiomass = {
#     'min': 0,
#     'max': 30,
#     'palette': ['lightyellow', 'lightgreen', 'green', 'darkgreen']
# }


# Map.centerObject(roi)
# Map.addLayer(
#     pred_simplified,
#     visPredictedBiomass,
#     "im1",)

# Map

In [14]:
# Export 2017
task1 = ee.batch.Export.image.toDrive(
    image=classified,
    description='Prediccion_2024_costa_2',
    folder='CH-delta',       # Carpeta en tu Google Drive
    fileNamePrefix='Prediccion_2024_costa_2',
    region=roi,
    scale=10,                    # Resolución de 100 metros
    crs='EPSG:4326',              # Puedes ajustar el CRS según tus datos
    maxPixels=1e13
)
task1.start()

print("🚀 Exportaciones lanzadas: revisa la pestaña 'Tasks' en GEE")

🚀 Exportaciones lanzadas: revisa la pestaña 'Tasks' en GEE
